# TP 2

## Imports

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import matplotlib
import plotly as py
import plotly.express as px 
import matplotlib.ticker as ticker
plt.style.use('default')
import plotly.offline as opy
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings('ignore')

## lectura de datos

In [18]:
entrenamiento = pd.read_csv('Training.csv')
entrenamiento.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,None,Netherlands,Account_Name_619,Opportunity_Name_12598,0,None,Person_Name_51,Person_Name_18,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/7/2015,None,None,None,None,None,None,NaT,1/30/2016,6/13/2016,Person_Name_18,Product_Family_77,Product_Name_99,EUR,0.52,USD,0.59,5/1/2016,6/30/2016,2016 - 5,Q2,2016,NaT,10,EUR,"5,272,800.00",EUR,"5,272,800.00",Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,None,Netherlands,Account_Name_619,Opportunity_Name_12600,1,39,Person_Name_51,Person_Name_20,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/7/2015,None,None,None,None,None,None,NaT,1/20/2016,1/15/2016,Person_Name_20,Product_Family_77,Product_Name_100,EUR,0.53,USD,0.60,1/18/2016,1/20/2016,2016 - 1,Q1,2016,NaT,0,EUR,"48,230.00",EUR,"48,230.00",Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,Source_7,United States,Account_Name_1794,Opportunity_Name_469,2,760,Person_Name_64,Person_Name_8,Account_Type_5,Opportunity_Type_1,Non Binding,Delivery_Terms_4,12/8/2015,None,None,None,None,None,None,NaT,9/30/2016,9/29/2016,Person_Name_8,Product_Family_81,Product_Name_91,USD,0.48,USD,0.48,1/25/2016,1/25/2016,2016 - 1,Q1,2016,NaT,0,USD,"83,865.60",USD,"83,865.60",Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,United States,Account_Name_1201,Opportunity_Name_415,3,None,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,12/8/2015,Other,Product_Type_0,Size_4,Product_Category_B_16,0.42,USD,NaT,NaT,3/27/2018,Person_Name_8,Product_Family_209,Product_Name_432,USD,0.53,USD,0.53,2/1/2018,3/31/2018,2018 - 2,Q1,2018,NaT,14,USD,"7,421,881.50",USD,"7,421,881.50",Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,United States,Account_Name_1201,Opportunity_Name_851,4,None,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,12/8/2015,Other,Product_Type_0,Size_4,Product_Category_B_16,0.42,USD,NaT,6/2/2017,3/27/2018,Person_Name_8,Product_Family_209,Product_Name_432,USD,0.53,USD,0.53,2/1/2018,2/28/2018,2018 - 2,Q1,2018,NaT,25,USD,"13,357,192.50",USD,"13,357,192.50",Closed Lost,Prod_Category_A_None


In [19]:
testeo = pd.read_csv('Test.csv')
testeo.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.33,USD,0.37,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,"162,240.00",EUR,"367,419.00",Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.32,USD,0.36,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,"78,624.00",EUR,"367,419.00",Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.28,USD,0.31,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,"126,555.00",EUR,"367,419.00",Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.33,USD,0.37,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,"243,360.00",EUR,"757,783.50",Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.32,USD,0.36,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,"157,248.00",EUR,"757,783.50",Prod_Category_A_None


## prosesamiento de los df

In [22]:
def convertirATiempo(df,lista):
    for columna in lista:
        df[columna] = pd.to_datetime(df[columna])
    return df

def convertirATipo(df,lista,tipo):
    for columna in lista:
        df[columna] = df[columna].astype(tipo)
    return df

def convertirAFlotante(df,lista):
    for columna in lista:
        df[columna] = df[columna].apply(lambda x: None if (x == "None" or x == "Other") else float(x))
    return df

def establecerTipos(df):
    """cambia los tipos de las columnas"""
    
    #fechas
    listaDeColumnasFechas = ['Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Last_Modified_Date','Delivery_Year','Planned_Delivery_End_Date','Planned_Delivery_Start_Date']
    df = convertirATiempo(df,listaDeColumnasFechas)
    #enteros
    listaDeColumnasEnteras = ['ID','Opportunity_ID']
    convertirATipo(df,listaDeColumnasEnteras,'int')
    #decimales
    listaDeColumnasEnteras = ['Price','ASP','ASP_(converted)','Total_Taxable_Amount','Total_Amount']
    convertirAFlotante(df,listaDeColumnasEnteras)
    #booleanos
    listaDeColumnasBooleanas = ['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved','Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved']
    convertirATipo(df,listaDeColumnasBooleanas,'bool')
    #categoricas
    listaDeColumnasCategoricas = ['Product_Category_B','Product_Family','Region','Territory','Bureaucratic_Code','Source ','Billing_Country','Account_Name','Opportunity_Name']
    convertirATipo(df,listaDeColumnasCategoricas,'category')
    return df

def eliminarColumnasSinImportancia(df):
    #"Last_Activity" y "Actual_Delivery_Date" son puros nulos
    #"Submitted_for_Approval" puros valores nulos
    #"Prod_Category_A" tiene todos sus valores iguales
    df = df.drop(['Submitted_for_Approval','Last_Activity','Actual_Delivery_Date','Prod_Category_A'], axis=1)
    return df

def agregarColumnas(df):
    df['Closed'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': True, 'Negotiation':False,'Proposal':False,'Qualification':False})#"Source " columna con un espacio en el nombre
    df['Won'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': False, 'Negotiation':False,'Proposal':False,'Qualification':False})#"Source " columna con un espacio en el nombre
    return df

#def unaFilaUnaOportunidad(df):
#    """se supone que en el df original puede haber mas de una fila por oportunidad de negocio"""
#    return df

def correccionDeTerritorios(df):
    #se refiere a diferentes partes de usa
    df = df.replace({'Territory':{'Central US':'USA','NE America':'USA', 'NW America':'USA','SW America':'USA','SE America':'USA'}})
    #se refiere a diferentes partes de usaSaudi Arabia
    df = df.replace({'Territory':{'KSA':'Saudi Arabia'}})
    #No se ha encontrado la dorma de los siguientes datos 'Kosovo','South America', 'South East Asia', 'UAE (Dubai)', 'UAE (Others)' (en el caso de kosovo no lo grafica)
    return df

def preparacionDelDataFrame(df):
    df = eliminarColumnasSinImportancia(df)
    df = establecerTipos(df)
    #df = unaFilaUnaOportunidad(df)
    df = correccionDeTerritorios(df)
    df = agregarColumnas(df)
    df.rename(columns = {'Source ':'Source'}, inplace = True)
    return df

In [12]:
def agrupacionDeOportunidades(df):
  df['modificaciones'] = 1
  data = df.groupby('Opportunity_ID').agg({'Won':'sum','Closed':'sum','modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           'Price':'mean','Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first'})#hay que seguir agregando cosas que son importantes
  data['Closed'] = data['Closed'].agg(lambda x: 1 if x > 0 else 0)
  data['Won'] = data['Won'].agg(lambda x: 1 if x > 0 else 0)
  data['Lost'] = data['Closed'] - data['Won']#si es won es closed pero si es closed y no es won es perdido
  data['Lost'] = data['Lost'].agg(lambda x: 1 if x > 0 else 0)

  data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: 1 if x > 0 else 0)
  data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: 1 if x > 0 else 0)
  #data = data.replace({'Won':})
  data = data[data['Closed'] > 0]#no interesan los que no se cerraron
  return data

## Ejecucion de cambios

In [20]:
entrenamiento = preparacionDelDataFrame(entrenamiento)
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16947 non-null  int64         
 1   Region                              16947 non-null  category      
 2   Territory                           16947 non-null  object        
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  bool          
 4   Pricing, Delivery_Terms_Approved    16947 non-null  bool          
 5   Bureaucratic_Code_0_Approval        16947 non-null  bool          
 6   Bureaucratic_Code_0_Approved        16947 non-null  bool          
 7   Bureaucratic_Code                   16947 non-null  category      
 8   Account_Created_Date                16947 non-null  datetime64[ns]
 9   Source                              16947 non-null  category      
 10  Billing_Country       

In [21]:
testeo['Stage'] = False
testeo = preparacionDelDataFrame(testeo)
testeo.info()

TypeError: ignored